# Import and combine raw datasets

In [1]:
import numpy as np
import pandas as pd
import os
import pickle
from joblib import Parallel, delayed
import re

We will import data from 3 distinct sources and attempt to combine them to generate the most complete dataset that we can.

In [15]:
d = './raw_data/'

#### Import: https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset/data?select=movies_metadata.csv

In [92]:
filename_list = ['movies_metadata.csv', 'credits.csv', 'keywords.csv']

movie_meta = pd.read_csv(d + "rounakbanik/movies_metadata.csv", 
                         converters={'id': lambda x: pd.to_numeric(x, errors='coerce'),
                                       'budget': lambda x: pd.to_numeric(x, errors='coerce'),
                                    'release_date': lambda x: pd.to_datetime(x, format='%Y-%m-%d', errors = 'coerce')})
credits = pd.read_csv(d + "rounakbanik/credits.csv",
                           converters={'id': lambda x: pd.to_numeric(x, errors='coerce')})

enhanced_boxoffice = pd.read_csv(d + "aditya126/enhanced_box_office_data(2000-2024)u.csv")

/var/folders/_q/50__mrnj2zv6hb8crhk3q46m0000gn/T/ipykernel_6944/3591271919.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_meta = pd.read_csv(d + "rounakbanik/movies_metadata.csv",


In [90]:
movie_meta.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862.0,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844.0,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602.0,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357.0,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0.0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862.0,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [91]:
# Inspect data entries
movie_meta.info()
movie_meta.describe(include = "all").T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   adult                  45466 non-null  object        
 1   belongs_to_collection  4494 non-null   object        
 2   budget                 45463 non-null  float64       
 3   genres                 45466 non-null  object        
 4   homepage               7782 non-null   object        
 5   id                     45463 non-null  float64       
 6   imdb_id                45449 non-null  object        
 7   original_language      45455 non-null  object        
 8   original_title         45466 non-null  object        
 9   overview               44512 non-null  object        
 10  popularity             45461 non-null  object        
 11  poster_path            45080 non-null  object        
 12  production_companies   45463 non-null  object        
 13  p

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
adult,45466,5,False,45454,NaN,NaN,NaN,NaN,NaN,NaN,NaN
belongs_to_collection,4494,1698,"{'id': 415931, 'name': 'The Bowery Boys', 'pos...",29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
budget,45463.0,NaN,NaN,NaN,4224578.813475,0.0,0.0,0.0,0.0,380000000.0,17424132.634341
genres,45466,4069,"[{'id': 18, 'name': 'Drama'}]",5000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
homepage,7782,7673,http://www.georgecarlin.com,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id,45463.0,NaN,NaN,NaN,108359.918813,2.0,26449.5,60003.0,157328.0,469172.0,112460.749278
imdb_id,45449,45417,tt1180333,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
original_language,45455,92,en,32269,NaN,NaN,NaN,NaN,NaN,NaN,NaN
original_title,45466,43373,Hamlet,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
overview,44512,44307,No overview found.,133,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Merge movie meta data and credits, and remove duplicates

In [79]:
merged_raw_nodups = movie_meta.drop_duplicates(subset = "id").merge(credits.drop_duplicates(subset = "id"), on='id').drop_duplicates(subset = "imdb_id")

In [9]:
d_raw = './raw_data/rajugc/'
filename_list = os.listdir(d_raw)
imported_csv_list = []

for filename in filename_list:
    imported = pd.read_csv(d_raw + filename)
    imported['rajugc_genre'] = filename[:-4]
    imported_csv_list.append(imported)

In [10]:
stacked_raw = pd.DataFrame(np.vstack(imported_csv_list), columns = imported_csv_list[0].columns.tolist())
stacked_raw_nodups =stacked_raw.drop_duplicates(subset = "movie_id")

In [11]:
stacked_raw_nodups.head()

,movie_id,movie_name,year,certificate,runtime,genre,rating,description,director,director_id,star,star_id,votes,gross(in $),rajugc_genre
0,tt5433140,Fast X,2023,NaN,NaN,"Action, Crime, Mystery",NaN,Dom Toretto and his family are targeted by the...,Louis Leterrier,/name/nm0504642/,"Vin Diesel, \nJordana Brewster, \nTyrese Gibso...","/name/nm0004874/,/name/nm0108287/,/name/nm0879...",NaN,NaN,crime
1,tt11564570,Glass Onion,2022,PG-13,139 min,"Comedy, Crime, Drama",7.2,Famed Southern detective Benoit Blanc travels ...,Rian Johnson,/name/nm0426059/,"Daniel Craig, \nEdward Norton, \nKate Hudson, ...","/name/nm0185819/,/name/nm0001570/,/name/nm0005...",333315.0,NaN,crime
2,tt10365998,Infinity Pool,2023,R,117 min,"Crime, Horror, Mystery",6.5,James and Em Foster are enjoying an all-inclus...,Brandon Cronenberg,/name/nm0188722/,"Alexander Skarsgård, \nMia Goth, \nCleopatra C...","/name/nm0002907/,/name/nm5301405/,/name/nm1671...",6955.0,NaN,crime
3,tt14138650,The Pale Blue Eye,2022,R,128 min,"Crime, Horror, Mystery",6.6,A world-weary detective is hired to investigat...,Scott Cooper,/name/nm0178376/,"Christian Bale, \nHarry Melling, \nSimon McBur...","/name/nm0000288/,/name/nm0577982/,/name/nm0564...",85087.0,NaN,crime
4,tt15789492,Infiesto,2023,TV-MA,96 min,"Crime, Drama, Thriller",5.5,Two detectives are called to a small mining to...,Patxi Amezcua,/name/nm0025538/,"Isak Férriz, \nIria del Río, \nAntonio Buíl, \...","/name/nm1929945/,/name/nm4170579/,/name/nm0125...",2081.0,NaN,crime


In [12]:
print(stacked_raw_nodups.info()) 

<class 'pandas.core.frame.DataFrame'>
Index: 243197 entries, 0 to 368299
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   movie_id      243197 non-null  object
 1   movie_name    243195 non-null  object
 2   year          204294 non-null  object
 3   certificate   55092 non-null   object
 4   runtime       159718 non-null  object
 5   genre         243197 non-null  object
 6   rating        138691 non-null  object
 7   description   243197 non-null  object
 8   director      222358 non-null  object
 9   director_id   222358 non-null  object
 10  star          198858 non-null  object
 11  star_id       204115 non-null  object
 12  votes         138693 non-null  object
 13  gross(in $)   12008 non-null   object
 14  rajugc_genre  243197 non-null  object
dtypes: object(15)
memory usage: 29.7+ MB
None


In [13]:
stacked_raw_nodups.describe().T

,count,unique,top,freq
movie_id,243197,243197,tt5433140,1
movie_name,243195,213572,Alone,35
year,204294,216,2022,7578
certificate,55092,42,Not Rated,16596
runtime,159718,365,90 min,11605
genre,243197,1745,Horror,16050
rating,138691.0,91.0,6.2,5050.0
description,243197,186759,Add a Plot,54449
director,222358,103466,Godfrey Ho,120
director_id,222358,96485,/name/nm0061792/,131


In [14]:
stacked_raw_nodups.loc[stacked_raw_nodups.movie_name == "Alone"]

,movie_id,movie_name,year,certificate,runtime,genre,rating,description,director,director_id,star,star_id,votes,gross(in $),rajugc_genre
27447,tt3460100,Alone,X,NaN,NaN,"Action, Crime, Drama",NaN,Ivy (Alyssa Arellano) is a very shy girl. She ...,"Alyssa Arellano, \nNatalie Camarillo",/name/nm6210735/,"Natalie Camarillo, \nAlyssa Arellano","/name/nm6210717/,/name/nm6210717/,/name/nm6210...",NaN,NaN,crime
36257,tt7711170,Alone,2020,R,98 min,"Drama, Horror, Mystery",6.2,A recently widowed traveler is kidnapped by a ...,John Hyams,/name/nm0404882/,"Jules Willcox, \nMarc Menchaca, \nAnthony Heal...","/name/nm2663082/,/name/nm0578766/,/name/nm0372...",22009.0,NaN,horror
37235,tt10192566,Alone,2020,R,92 min,"Horror, Thriller",5.0,"When an outbreak hits, Aidan barricades himsel...",Johnny Martin,/name/nm0552565/,"Tyler Posey, \nSummer Spiro, \nDonald Sutherla...","/name/nm0692677/,/name/nm4887855/,/name/nm0000...",3826.0,NaN,horror
42184,tt0484090,Alone,2007,NaN,105 min,"Drama, Horror, Thriller",6.4,Deceased Siamese twin returns to haunt survivi...,"Banjong Pisanthanakun, \nParkpoom Wongpoom",/name/nm1802294/,"Marsha Wattanapanich, \nVittaya Wasukraipaisan...","/name/nm1059740/,/name/nm2589815/,/name/nm2297...",5509.0,NaN,horror
43121,tt4271730,Alone,2015,Not Rated,133 min,Horror,3.7,A supernatural mystery about a young woman who...,Bhushan Patel,/name/nm1022024/,"Bipasha Basu, \nKaran Singh Grover, \nSulabha ...","/name/nm0060607/,/name/nm2822547/,/name/nm0038...",1778.0,NaN,horror
46376,tt0263053,Alone,2002,R,88 min,"Horror, Mystery, Thriller",4.3,Experience the fear as you're sucked into the ...,Phil Claydon,/name/nm0165616/,"Miriam Margolyes, \nJohn Shrapnel, \nLaurel Ho...","/name/nm0546816/,/name/nm0795344/,/name/nm0391...",651.0,NaN,horror
51586,tt1800240,Alone,2013,Unrated,84 min,"Horror, Mystery, Thriller",3.5,An agoraphobic man trapped in his own home is ...,Rory Douglas Abel,/name/nm2109871/,"Michael Jefferson, \nEmma Dubery, \nSarah Scho...","/name/nm3959928/,/name/nm2576494/,/name/nm3600...",237.0,NaN,horror
56210,tt15314332,Alone,LXII,NaN,NaN,"Horror, Thriller",NaN,Isolation leads to insanity as an ominous shad...,"Mandi Mellen, \nDan Salem",/name/nm4593457/,"Mandi Mellen, \nDan Salem, \nEmily Jira, \nPri...","/name/nm4568135/,/name/nm4593457/,/name/nm4568...",NaN,NaN,horror
56580,tt26653746,Alone,LXXVII,NaN,NaN,Horror,NaN,Add a Plot,NaN,NaN,NaN,NaN,NaN,NaN,horror
57980,tt1084944,Alone,I,NaN,NaN,"Horror, Thriller",NaN,"A woman who, after she suffers a tragedy, beco...",NaN,NaN,NaN,NaN,NaN,NaN,horror
